In [43]:
import random
import numpy as np

In [59]:
area = np.zeros((10,10),dtype=float)

AP_num = np.random.poisson(lam=3.0, size=None)
# MT_num = AP_num
MT_num = np.random.poisson(lam=5.0, size=None)
# AP_num = 2
# MT_num = 2
좌표 = []
area_dasi = area

## AP 발생
for k in range(AP_num) :
  x= np.random.randint(10,size=1) # AP의 위치
  y= np.random.randint(10,size=1)
  while True : # 겹치면 안됨. 
    if area_dasi[x,y] !=0 :
       x= np.random.randint(10,size=1) # AP의 위치
       y= np.random.randint(10,size=1)
       
    else : 
       break
  area_dasi[x,y] = 1 

## MT 발생
for k in range(MT_num) :
  x_dasi= np.random.randint(10,size=1) # MT의 위치
  y_dasi= np.random.randint(10,size=1)
  while True :
    if area_dasi[x_dasi,y_dasi]!=0 : # AP와 MT가 이미 있으면 다시 선언
      x_dasi= np.random.randint(10,size=1)
      y_dasi= np.random.randint(10,size=1)
    else :
      break
  area_dasi[x_dasi,y_dasi] = -1 

## 각 MT 마다 가장 가까운 AP를 구하는 식
for a in range(MT_num) :
  x_= np.where(area_dasi == -1)[0][a]
  y_= np.where(area_dasi == -1)[1][a]
  min = 10000
  x_좌표 = None
  y_좌표 = None
  for k in range(AP_num) : 
    i = np.where(area_dasi == 1)[0][k]
    j = np.where(area_dasi == 1)[1][k]
    cal = (i-x_)**2 + (j-y_)**2
    if min > cal :
      min = cal
      x_좌표 = i
      y_좌표 = j
  좌표.append([(x_,y_),(x_좌표,y_좌표)])

In [60]:
area_dasi

array([[-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [61]:
좌표

[[(0, 0), (1, 0)], [(1, 2), (2, 1)], [(5, 3), (4, 3)], [(8, 1), (6, 1)]]

In [62]:
index =np.where(area_dasi == 1)
AP=[]
for i in range(AP_num) :
  x_index = index[0][i]
  y_index = index[1][i]
  AP.append([x_index,y_index])

In [63]:
AP

[[1, 0], [2, 1], [4, 3], [6, 1], [6, 5], [6, 7]]

In [64]:
AP_num

6

In [65]:
MT_num

4

In [67]:
allocation_AP = {}
for i in range(AP_num) :
  for j in 좌표 :
    if tuple(AP[i]) == j[1] :
      if tuple(AP[i]) in allocation_AP : # 키가 있으면
        allocation_AP[tuple(AP[i])].append(j[0])
      else : # 키가 없으면
        allocation_AP[tuple(AP[i])] = [j[0]]
#a.get('a')

In [68]:
allocation_AP

{(1, 0): [(0, 0)], (2, 1): [(1, 2)], (4, 3): [(5, 3)], (6, 1): [(8, 1)]}

In [69]:
# ALOHA의 경우
p = 0.3
for i in range(AP_num) :
  tran = 0
  if tuple(AP[i]) in allocation_AP :
    tran = len(allocation_AP[tuple(AP[i])])*p*(1-p)**(len(allocation_AP[tuple(AP[i])])-1)
  print('{}AP(위치 {})의 ALOHA transmission capacity : {}'.format(i+1,AP[i], tran))

1AP(위치 [1, 0])의 ALOHA transmission capacity : 0.3
2AP(위치 [2, 1])의 ALOHA transmission capacity : 0.3
3AP(위치 [4, 3])의 ALOHA transmission capacity : 0.3
4AP(위치 [6, 1])의 ALOHA transmission capacity : 0.3
5AP(위치 [6, 5])의 ALOHA transmission capacity : 0
6AP(위치 [6, 7])의 ALOHA transmission capacity : 0


In [70]:
# 조금 rough하게 가정해보자.
# slotted aloha의 경우 100 bit 짜리를 100sec에 나누어서 보냈다고 치면 100 * N * p * (1-p)**(N-1) / 100 이 bps/area 가 나온다.
# 이를 이용해서 CSMA/CA의 경우에는 먼저 RTS를 보내므로 RTS와 CTS 의 크기가 20 , 데이터의 크기가 100 bit 라고 하면 
# RTS만 ALOHA 기법 처럼 사용되고 나머지 초 단위에서는 백퍼센트 성공이 이루어진다.그래서 식은
# 20 * N * p * (1-p)**(N-1) + 100 * 1 / 120 이 bps/area가 될 것으로 예상할 수 있다.
# 여기서는 rough하게 가정했기 때문에 BEB를 고려하지 않았는데 상관없는 것이 어짜피 slotted ALOHA에서도 충돌이 일어나면 BEB 기법을 사용하기 때문에 고려하지 않아도 된다.(위에서 고려하지 않았기 떄문에)

p = 0.3
for i in range(AP_num) :
  tran = 0
  if tuple(AP[i]) in allocation_AP :
    tran = (20*len(allocation_AP[tuple(AP[i])])*p*(1-p)**(len(allocation_AP[tuple(AP[i])])-1)+100)/120
  print('{}AP(위치 {})의 CSMA/CA transmission capacity : {}'.format(i+1,AP[i], tran))

# 역시 CSMA/CA 가 더 capacity가 큰 것을 확인할 수 있다.

1AP(위치 [1, 0])의 CSMA/CA transmission capacity : 0.8833333333333333
2AP(위치 [2, 1])의 CSMA/CA transmission capacity : 0.8833333333333333
3AP(위치 [4, 3])의 CSMA/CA transmission capacity : 0.8833333333333333
4AP(위치 [6, 1])의 CSMA/CA transmission capacity : 0.8833333333333333
5AP(위치 [6, 5])의 CSMA/CA transmission capacity : 0
6AP(위치 [6, 7])의 CSMA/CA transmission capacity : 0


In [38]:
# CSMA/CA의 경우도 RTS를 위의 ALOHA 처럼 random하게 보낼 것인데 그 크기가 전체 데이터 보다 작으므로 p의 값이 더 작을 것으로 예상할 수 있다.
# 무엇보다 하나의 time slot에서 보자면 slotted ALOHA와 같은 단위라고 봤을 때 데이터의 크기보다 훨씬 작은 크기의 RTS이므로 거의 충돌이 일어나지 않는다고 가정하였다.
# 그래서 그냥 RTS가 오고 나서의 확률을 보자면 사실 CSMA/CA의 경우에는 한 time slot에서 항상 보낼 수 있으므로 1의 크기를 가진다. 
# 그렇기 때문에 매우 rough한 가정인 것을 알 수 있어서 조금 동작의 세밀함이 필요하다.
# 그래서 그냥 강의 노트에 있는 sensing Threshold 기법을 사용해 볼 수 있을 것이다. 근데 아쉬운 점은 너무 변수가 많아서 그 값들을 설정하는 것이 여이치 않아서 위의 값으로 대신하겠다.

lamda = 5
tau = 1
R_s = 1
beta = 1 # target SIR
r_t = 1
p_s = np.exp((np.exp(-lamda*tau*3.141592*R_s**2)-1)/(R_s**2) * np.sqrt(beta)*r_t**2 * np.arctan(np.sqrt(beta)*r_t**2/(R_s**2)))

In [39]:
p_s

0.4559381817314047